In [152]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA, RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.vectorstores import Pinecone
from pinecone import Pinecone, ServerlessSpec
import pinecone
from langchain.pinecone import PineconeVectorStore
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate 
from langchain.llms import CTransformers
from transformers import pipeline, AutoModel
from langchain.llms.huggingface_pipeline import HuggingFacePipeline 

In [45]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                   glob="*.pdf",
                   loader_cls=PyPDFLoader)

    documents = loader.load()

    return documents

In [46]:
extracted_data = load_pdf("data/")

In [47]:
# extracted_data

In [48]:
# Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [49]:
text_chunks = text_split(extracted_data)
print("length of chunks: ", len(text_chunks))

length of chunks:  7020


In [50]:
# download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [51]:
embeddings = download_hugging_face_embeddings()

In [52]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [53]:
# test for "hello world"
query_result = embeddings.embed_query("Hello world")
print("Query Length: ", len(query_result))

Query Length:  384


In [54]:
query_result

[-0.03447727486491203,
 0.031023237854242325,
 0.006734995637089014,
 0.026108987629413605,
 -0.039362020790576935,
 -0.1603025197982788,
 0.06692394614219666,
 -0.006441484205424786,
 -0.04745049774646759,
 0.014758791774511337,
 0.07087524980306625,
 0.05552758276462555,
 0.01919332891702652,
 -0.026251375675201416,
 -0.010109515860676765,
 -0.02694043703377247,
 0.0223073810338974,
 -0.022226618602871895,
 -0.1496926099061966,
 -0.017493048682808876,
 0.0076762172393500805,
 0.054352305829524994,
 0.003254448063671589,
 0.03172590956091881,
 -0.0846213772892952,
 -0.02940598502755165,
 0.0515955314040184,
 0.0481240339577198,
 -0.0033148115035146475,
 -0.058279164135456085,
 0.04196930676698685,
 0.022210676223039627,
 0.1281888335943222,
 -0.02233891747891903,
 -0.011656291782855988,
 0.06292839348316193,
 -0.03287632763385773,
 -0.09122605621814728,
 -0.03117537684738636,
 0.05269954726099968,
 0.047034818679094315,
 -0.0842030867934227,
 -0.030056215822696686,
 -0.020744841545820

In [55]:
import os
PINECONE_API_KEY = "c92f6e26-04c2-4ac5-9fb7-54e572254d5d"
PINECONE_API_ENV = "gcp-starter"

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['PINECONE_API_ENV'] = PINECONE_API_ENV

In [56]:
text_field = [text.page_content for text in text_chunks]
index_name = "medical-chatbot"

# vectorstore = PineconeVectorStore.from_texts(
#     text_field, embeddings, index_name=index_name
# )

In [57]:
# lets query
docsearch = PineconeVectorStore.from_existing_index(index_name,embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k=3)

print('Result', docs)

Result [Document(page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and shellfish• eggs• wheat• milk• food additives and preservatives\nThe following types of drugs commonly ca

In [58]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you dont't know the answer, just say that you don't know. Don't try to make up an answer

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else:
Helpful answer: 
"""

In [59]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [60]:
llm = CTransformers(model='model/llama-2-7b-chat.ggmlv3.q4_0.bin',
                    model_type='llama',
                    config={'max_new_tokens':512,
                            'temperature': 0.8})

In [169]:
# from __future__ import annotations

# import warnings
# from abc import ABC, abstractmethod
# from inspect import signature
# from typing import TYPE_CHECKING, Any, Dict, List, Optional

# from langchain_core.retrievers import BaseRetriever
# from langchain_core.callbacks import CallbackManagerForRetrieverRun, AsyncCallbackManagerForRetrieverRun
# from langchain_core.documents import Document
# from langchain_core.load.dump import dumpd
# from langchain_core.runnables import (
#     Runnable,
#     RunnableConfig,
#     RunnableSerializable,
#     ensure_config,
# )
# if TYPE_CHECKING:
#     from langchain_core.callbacks.manager import (
#         AsyncCallbackManagerForRetrieverRun,
#         CallbackManagerForRetrieverRun,
#         Callbacks,
#     )
# from langchain_core.runnables.config import run_in_executor
# from abc import abstractmethod
# from typing import List


# class CustomRetriever(BaseRetriever):
    
#     def _get_relevant_documents(
#         self,
#         query: str,
#         *,
#         callbacks: Callbacks = None,
#         tags: Optional[List[str]] = None,
#         metadata: Optional[Dict[str, Any]] = None,
#         run_name: Optional[str] = None,
#         **kwargs: Any,
#     ) -> List[Document]:
#         """Retrieve documents relevant to a query.
#         Args:
#             query: string to find relevant documents for
#             callbacks: Callback manager or list of callbacks
#             tags: Optional list of tags associated with the retriever. Defaults to None
#                 These tags will be associated with each call to this retriever,
#                 and passed as arguments to the handlers defined in `callbacks`.
#             metadata: Optional metadata associated with the retriever. Defaults to None
#                 This metadata will be associated with each call to this retriever,
#                 and passed as arguments to the handlers defined in `callbacks`.
#         Returns:
#             List of relevant documents
#         """
#         from langchain_core.callbacks.manager import CallbackManager

#         callback_manager = CallbackManager.configure(
#             callbacks,
#             None,
#             verbose=kwargs.get("verbose", False),
#             inheritable_tags=tags,
#             local_tags=self.tags,
#             inheritable_metadata=metadata,
#             local_metadata=self.metadata,
#         )
#         run_manager = callback_manager.on_retriever_start(
#             dumpd(self),
#             query,
#             name=run_name,
#         )
#         try:
#             _kwargs = kwargs if self._expects_other_args else {}
#             if self._new_arg_supported:
#                 result = self._get_relevant_documents(
#                     query, run_manager=run_manager, **_kwargs
#                 )
#             else:
#                 result = self._get_relevant_documents(query, **_kwargs)
#         except Exception as e:
#             run_manager.on_retriever_error(e)
#             raise e
#         else:
#             run_manager.on_retriever_end(
#                 result,
#             )
#             return result


#     async def _aget_relevant_documents(
#         self,
#         query: str,
#         *,
#         callbacks: Callbacks = None,
#         tags: Optional[List[str]] = None,
#         metadata: Optional[Dict[str, Any]] = None,
#         run_name: Optional[str] = None,
#         **kwargs: Any,
#     ) -> List[Document]:
#         """Asynchronously get documents relevant to a query.
#         Args:
#             query: string to find relevant documents for
#             callbacks: Callback manager or list of callbacks
#             tags: Optional list of tags associated with the retriever. Defaults to None
#                 These tags will be associated with each call to this retriever,
#                 and passed as arguments to the handlers defined in `callbacks`.
#             metadata: Optional metadata associated with the retriever. Defaults to None
#                 This metadata will be associated with each call to this retriever,
#                 and passed as arguments to the handlers defined in `callbacks`.
#         Returns:
#             List of relevant documents
#         """
#         from langchain_core.callbacks.manager import AsyncCallbackManager

#         callback_manager = AsyncCallbackManager.configure(
#             callbacks,
#             None,
#             verbose=kwargs.get("verbose", False),
#             inheritable_tags=tags,
#             local_tags=self.tags,
#             inheritable_metadata=metadata,
#             local_metadata=self.metadata,
#         )
#         run_manager = await callback_manager.on_retriever_start(
#             dumpd(self),
#             query,
#             name=run_name,
#         )
#         try:
#             _kwargs = kwargs if self._expects_other_args else {}
#             if self._new_arg_supported:
#                 result = await self._aget_relevant_documents(
#                     query, run_manager=run_manager, **_kwargs
#                 )
#             else:
#                 result = await self._aget_relevant_documents(query, **_kwargs)
#         except Exception as e:
#             await run_manager.on_retriever_error(e)
#             raise e
#         else:
#             await run_manager.on_retriever_end(
#                 result,
#             )
#             return result



In [226]:
from typing import TYPE_CHECKING, Any, Dict, List, Optional
from langchain.vectorstores import VectorStore
from langchain_core.documents import Document
from langchain_core.callbacks import CallbackManagerForRetrieverRun, AsyncCallbackManagerForRetrieverRun
from dataclasses import dataclass

class CustomRetriever(BaseRetriever):
    
    def __init__(self):
        self.retriever: docsearch.as_retriever(search_kwargs={"k": 2})
        vectorstores: VectorStore

    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        
        documents = self.retriever.get_relevant_documents(query, callbacks=run_manager.get_child())
        
        documents = sorted(documents, key=lambda doc: doc.metadata.get('source'))
        
        return documents

    async def _aget_relevant_documents(
        self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    ) -> List[Document]:
        
        documents = self.retriever.get_relevant_documents(query, callbacks=run_manager.get_child())
        
        documents = sorted(documents, key=lambda doc: doc.metadata.get('source'))
        
        return documents

In [198]:
# from langchain.schema.retriever import BaseRetriever, Document
# from langchain_core.vectorstores import VectorStoreRetriever
# from dataclasses import Field, field

# class CustomRetriever(VectorStoreRetriever):
#     vectorstore: VectorStoreRetriever
#     search_type: str = "similarity"
#     search_kwargs: dict = Field(default_factory=dict)

#     def _get_relevant_documents(self, query: str) -> List[Document]:
#         results = self.vectorstore.get_relevant_documents(query=query)
#         return sorted(results, key=lambda doc: doc.metadata.get('source'))
    
#     async def _aget_relevant_documents(self, query: str) -> List[Document]:
#         results = self.vectorstore.get_relevant_documents(query=query)
#         return sorted(results, key=lambda doc: doc.metadata.get('source'))

In [199]:
# from langchain_core.vectorstores import VectorStoreRetriever
# VectorStoreRetriever(docsearch.as_retriever(search_kwargs={'k':2}))

In [227]:

# CustomRetriever(docsearch.as_retriever(search_kwargs={'k':2}))

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=CustomRetriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

AttributeError: __fields_set__

In [ ]:
while True:
    user_input = input(f"Input Prompt:")
    result = qa({'query': user_input})
    print("Response: ", result['result'] )